In [32]:
import pandas as pd
import os
import numpy as np
import pydicom
from skimage.io import imsave
import skimage
from tqdm import tqdm

In [33]:
# constants
#print(os.listdir(os.getcwd()))
data_path = 'Prostate-MRI-US-Biopsy'
target_png_dir = 'train_png'
if not os.path.exists(target_png_dir):
    os.makedirs(target_png_dir)

label_df = pd.read_csv('Target Data_2019-12-05.csv')
# display(label_df)

save_resize = 224

In [35]:
# dcm_names = []

# with default labeling, need at least 30 slices per volume 
# to reach required dataset size
for row_idx, row in tqdm(label_df.iterrows(), total=len(label_df)):
    patient_id = row['Patient ID']
    # found_ct = 0
    # find dicom for this row
    if found_ct == 0:
        for root, dirs, files in os.walk(os.path.join(data_path, patient_id), topdown=False):
            for name in files:
                full_name = os.path.join(root, name)
                if row['seriesInstanceUID_US'] in full_name:
                    # save pngs in dir according to score
                    score = str(row['UCLA Score (Similar to PIRADS v2)'])
                    png_out_dir = os.path.join(target_png_dir, score)
                    if not os.path.exists(png_out_dir):
                        os.makedirs(png_out_dir)
                    
                    # load dicom and extract slices from central
                    # half region
                    obj = pydicom.dcmread(full_name)
                    dcm_arr = obj.pixel_array
                    img_type = obj.PhotometricInterpretation
                    total_slices = dcm_arr.shape[0]
                    
                    # save each slice as png
                    for sample_slice in range(total_slices // 4, (3 * total_slices) // 4):
                        img = dcm_arr[sample_slice]
                        
                        # uint16 -> float, scaled properly for uint8
                        img = img.astype(float) * 255. / img.max()
                        # resize to size I will use for experiments
                        img = skimage.transform.resize(img, (save_resize, save_resize))
                        # float -> uint8
                        img = img.astype(np.uint8)
                        
                        if img_type == "MONOCHROME1":
                            img = np.invert(img)

                        # print(dcm_arr.shape)
                        png_path = '{}_{}.png'.format(row_idx, sample_slice)
                        png_path = os.path.join(png_out_dir, png_path)

                        imsave(png_path, img)

100%|███████████████████████████████████████| 1621/1621 [49:57<00:00,  1.85s/it]
